In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

import import_ipynb
from branch_adapter.classification.classification_network import CNN_classify
from branch_adapter.classification.classification_data import classification_data
import branch_adapter.classification.classification_model_utils as utils

importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_network.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_data_utils.ipynb
importing Jupyter notebook from D:\WETrak\branch_adapter\classification\classification_model_utils.ipynb


In [2]:
def classification_inference(data_path,user_name,runs_dir,classify_num):
    finger_type_list = ['Index','Middle','Ring']
    # -------------------------------------Gpu Device------------------------------------------------- #
    global device
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    # -------------------------------------Model Initialize-------------------------------------------------- #
    model = CNN_classify(classify_num=int(classify_num),finger_type=finger_type_list).cuda()
    
    # -------------------------------------Data Initialize--------------------------------------------------- #
    test_set = classification_data(data_path, user_name, "test", finger_type_list,classify_num)
    test_loader = DataLoader(test_set,batch_size= len(test_set), shuffle = False)
    
    # -------------------------------------Inference--------------------------------------------------- #
    checkpoint_dir = os.path.join(runs_dir, 'checkpoints')
    print("load classification checkpoingts from {}".format(os.path.abspath(checkpoint_dir)))
    checkpoint = utils.load_checkpoint(os.path.join(checkpoint_dir),"mean")
    model.load_state_dict(checkpoint['model_state_dict'])
    
    with torch.no_grad():
        model.eval()
        for i,(emg,gt) in enumerate(test_loader):
            emg = emg.unsqueeze(1)
            emg = utils.data_gpu(emg, device)
            features, out = model(emg)
            
            out = F.softmax(out,dim=1)
            prob_mse = out.cpu().numpy()
            
    return prob_mse

In [3]:
def get_cascade_inference(data_path,output_path,user_name,target):
    if target == 15:
        
        classify_id = {'15':"15",
                        '5':"5",
                        '3':"3"}

        root_dir = os.path.join(output_path,user_name,"classify")
        runs_dir,prob = {},{}

        for key, value in classify_id.items():
            runs_dir[key] = os.path.join(root_dir,value)

            prob[key] = classification_inference(data_path,user_name,runs_dir[key],int(key))
#             print("prob[key].shape:",prob[key].shape)
        
        prob_15,prob_5,prob_3 = prob['15'],prob['5'],prob['3']
        new_prob_15,new_prob_5,new_prob_3 = np.zeros(prob_15.shape),np.zeros(prob_15.shape),np.zeros(prob_15.shape)

        sub_target = 5
        interval = int(target/sub_target)
        for i in range(sub_target):
            new_prob_5[:, i*interval:i*interval+interval, :] = prob_5[:,i:i+1,:]

        sub_target = 3
        interval = int(target/sub_target)
        for i in range(sub_target):
            new_prob_3[:, i*interval:i*interval+interval, :] = prob_3[:,i:i+1,:]

        new_prob_15 = prob_15 + new_prob_5 + new_prob_3
        new_prob_15 = torch.tensor(new_prob_15)
        _, tags = torch.max(new_prob_15, dim = 1)
        
    return tags